In [ ]:
import os
from volcenginesdkarkruntime import Ark
base_url = "https://ark.cn-beijing.volces.com/api/v3"
api_key = (os.getenv("ARK_API_KEY") or "").strip()
ak = (os.getenv("VOLC_ACCESS_KEY_ID") or "").strip()
sk = (os.getenv("VOLC_SECRET_ACCESS_KEY") or "").strip()
client = Ark(base_url=base_url, api_key=api_key) if api_key else (Ark(base_url=base_url, ak=ak, sk=sk) if ak and sk else None)
print("豆包API: 已就绪", base_url, api_key, ak, sk, sep='\n') if client else print("豆包API: 未配置，请设置 ARK_API_KEY 或 VOLC_ACCESS_KEY_ID/VOLC_SECRET_ACCESS_KEY")

In [ ]:
# Import packages
from volcenginesdkarkruntime import Ark
from typing import List, Tuple
import gradio as gr
import json
import os

# ===== 设置豆包 Ark API Key =====
# TODO: 在此处填入你的豆包 Ark API Key
DOUBAO_API_KEY = os.environ.get("ARK_API_KEY")
if not DOUBAO_API_KEY:
    DOUBAO_API_KEY = input("请输入你的豆包 Ark API Key: ")

# ===== 初始化 Ark 客户端 =====
client = Ark(api_key=DOUBAO_API_KEY)

# ===== 检测豆包 API 是否配置成功 =====
try:
    # 发送一次简单请求 (例如使用 "doubao-seed-1-6-251015" 或你有权限的模型)
    response = client.chat.completions.create(
        model="doubao-seed-1-6-251015",
        messages=[
            {"role": "user", "content": "hello"}
        ]
    )

    print("豆包 API 设置成功!!")
    print("测试输出:", response.choices[0])

except Exception as e:
    print("豆包 API 配置似乎有问题，请检查 API Key 或模型名称")
    print("错误信息:", e)


In [ ]:
import os
from volcenginesdkarkruntime import Ark

# 请确保您已将 API Key 存储在环境变量 ARK_API_KEY 中
# 初始化Ark客户端，从环境变量中读取您的API Key
client = Ark(
    # 此为默认路径，您可根据业务所在地域进行配置
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    # 从环境变量中获取您的 API Key。此为默认方式，您可根据需要进行修改
    api_key=os.environ.get("ARK_API_KEY"),
)

response = client.chat.completions.create(
    # 指定您创建的方舟推理接入点 ID，此处已帮您修改为您的推理接入点 ID
    model="doubao-seed-1-6-251015",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://ark-project.tos-cn-beijing.ivolces.com/images/view.jpeg"
                    },
                },
                {"type": "text", "text": "这是哪里？"},
            ],
        }
    ],
    
)

print(response.choices[0])

## Part 3：使用豆包 - 文本对话示例
本节演示如何使用 Ark 客户端调用豆包模型进行纯文本对话。

In [ ]:
# ================================================
# 1. Import Packages
# ================================================
import os
import json
import traceback
from typing import List, Tuple
import gradio as gr

from volcenginesdkarkruntime import Ark

# ================================================
# 2. 自动读取 API Key / AKSK 并初始化 Ark client
# ================================================
BASE_URL = "https://ark.cn-beijing.volces.com/api/v3"

api_key = os.getenv("ARK_API_KEY", "").strip()
ak = os.getenv("VOLC_ACCESS_KEY_ID", "").strip()
sk = os.getenv("VOLC_SECRET_ACCESS_KEY", "").strip()

if api_key:
    client = Ark(api_key=api_key, base_url=BASE_URL)
elif ak and sk:
    client = Ark(ak=ak, sk=sk, base_url=BASE_URL)
else:
    raise ValueError("❌ 未检测到 ARK_API_KEY 或 AK/SK，请设置环境变量后再运行！")

# ================================================
# 3. 手动设置 endpoint_id（必须在控制台创建）
# ================================================
MODEL_ID = "ep-20251117212117-9ztrl"  # ←←← 这里填你的 endpoint_id

# ================================================
# 4. 测试豆包 API
# ================================================
def test_ark():
    print("正在测试豆包 Ark API...")
    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": "测试一下你是否正常？"}],
        )
        txt = resp.choices[0].message.content
        print("✅ API 正常工作")
        print("返回内容：", txt)
    except Exception:
        print("❌ API 测试失败")
        traceback.print_exc()

test_ark()

# ================================================
# 5. Summarization 逻辑
# ================================================
DEFAULT_PROMPT = "请总结下面的文章内容："

def reset() -> List:
    return []

def interact_summarization(prompt: str, article: str, temp=1.0):
    full_input = f"{prompt}\n\n{article}"

    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": full_input}],
            temperature=temp,
        )
        answer = resp.choices[0].message.content
        return [(full_input, answer)]
    except Exception:
        err = traceback.format_exc()
        return [(full_input, f"❌ 调用豆包 API 出错：\n{err}")]

def export_summarization(chatbot: List[Tuple[str, str]], article: str):
    data = {"chatbot": chatbot, "article": article}
    with open("part1.json", "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ================================================
# 6. 构建 Gradio UI
# ================================================
with gr.Blocks() as demo:
    gr.Markdown("# 使用豆包 Ark Summarization Demo")

    chatbot = gr.Chatbot()

    prompt_box = gr.Textbox(label="Prompt", value=DEFAULT_PROMPT, visible=False)
    article_box = gr.Textbox(label="输入文章", lines=10, interactive=True)

    temp_slider = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")

    with gr.Row():
        send_btn = gr.Button("发送")
        reset_btn = gr.Button("重置")

    export_btn = gr.Button("导出 JSON")

    send_btn.click(interact_summarization, [prompt_box, article_box, temp_slider], chatbot)
    reset_btn.click(reset, outputs=[chatbot])
    export_btn.click(export_summarization, inputs=[chatbot, article_box])

# ================================================
# 7. 启动 Gradio
# ================================================
demo.launch(debug=True, share=True)


In [ ]:
# load the conversation log json file
with open("part1.json", "r") as f:
    context = json.load(f)

chatbot = context['chatbot']
article = context['article']
summarization = chatbot[0][-1]

# This part constructs the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Part1: Summarization\nFill in any article you like and let the chatbot summarize it for you!!")
    chatbot = gr.Chatbot(value = context['chatbot'])
    article_textbox = gr.Textbox(label="Article", interactive = False, value = context['article'])
    with gr.Column():
        gr.Markdown("# Copy this part to the grading system.")
        gr.Textbox(label = "article", value = article, show_copy_button = True)
        gr.Textbox(label="summarization", value = summarization, show_copy_button = True)

demo.launch(debug = True)

In [ ]:
import json
import gradio as gr
from typing import List, Tuple
import os

# -----------------------------
# 1. 初始化豆包 Ark 客户端
# -----------------------------
from volcenginesdkarkruntime import Ark

api_key = os.getenv("ARK_API_KEY")
ak = os.getenv("VOLC_ACCESS_KEY_ID")
sk = os.getenv("VOLC_SECRET_ACCESS_KEY")
print(api_key, ak, sk)
if api_key:
    client = Ark(
        base_url="https://ark.cn-beijing.volces.com/api/v3",
        api_key=api_key
    )
elif ak and sk:
    print(ak, sk)
    client = Ark(
        base_url="https://ark.cn-beijing.volces.com/api/v3",
        ak=ak,
        sk=sk
    )
else:
    raise ValueError("请设置 ARK_API_KEY 或 VOLC_ACCESS_KEY_ID / VOLC_SECRET_ACCESS_KEY")

MODEL_NAME = "ep-20251117212117-9ztrl"

def test_ark():
    print("正在测试豆包 Ark API...")
    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": "测试一下你是否正常？"}],
        )
        txt = resp.choices[0].message.content
        print("✅ API 正常工作")
        print("返回内容：", txt)
    except Exception:
        print("❌ API 测试失败")
        traceback.print_exc()

test_ark()
# -----------------------------
# 2. 自定义角色描述
# -----------------------------
character_for_chatbot = "Mage Mentor"
prompt_for_roleplay = (
    "You are a powerful, wise mage mentor. "
    "Speak in an ancient, mystical tone, giving the user magical guidance."
)

# -----------------------------
# 3. 与豆包模型交互
# -----------------------------
def interact_roleplay(chatbot: List[Tuple[str, str]], user_input: str, temp=1.0):
    try:
        messages = [{
            "role": "system",
            "content": prompt_for_roleplay
        }]

        for user, bot in chatbot:
            messages.append({"role": "user", "content": user})
            messages.append({"role": "assistant", "content": bot})

        messages.append({"role": "user", "content": user_input})

        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=temp
        )

        raw_content = resp.choices[0].message.content
        answer = ""

        # 兼容：字符串形式
        if isinstance(raw_content, str):
            answer = raw_content

        # 兼容：多段 content 列表
        elif isinstance(raw_content, list):
            for c in raw_content:
                if isinstance(c, dict) and c.get("type") == "text":
                    answer += c.get("text", "")

        else:
            answer = str(raw_content)

        chatbot.append((user_input, answer))

    except Exception as e:
        print(f"Error occurred: {e}")
        chatbot.append((user_input, f"Error: {e}"))

    return chatbot

# -----------------------------
# 4. 重置
# -----------------------------
def reset():
    return []

# -----------------------------
# 5. 导出
# -----------------------------
def export_roleplay(chatbot, description):
    with open("part2.json", "w") as f:
        json.dump({"chatbot": chatbot, "description": description}, f,
                  ensure_ascii=False, indent=2)

# -----------------------------
# 6. Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("# Part2: Role Play")

    chatbot = gr.Chatbot()
    input_box = gr.Textbox(label="Your input")
    temp_slider = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")

    send_btn = gr.Button("Send")
    reset_btn = gr.Button("Reset")
    export_btn = gr.Button("Export")

    send_btn.click(interact_roleplay, inputs=[chatbot, input_box, temp_slider], outputs=[chatbot])
    reset_btn.click(reset, outputs=[chatbot])
    export_btn.click(export_roleplay, inputs=[chatbot, gr.Textbox(value=character_for_chatbot)])

demo.launch(debug=True, share=True)


In [ ]:
import json
import gradio as gr
from typing import List, Tuple
import os

# -----------------------------
# 1. 初始化豆包 Ark 客户端
# -----------------------------
from volcenginesdkarkruntime import Ark

api_key = os.getenv("ARK_API_KEY")
ak = os.getenv("VOLC_ACCESS_KEY_ID")
sk = os.getenv("VOLC_SECRET_ACCESS_KEY")

if api_key:
    client = Ark(
        base_url="https://ark.cn-beijing.volces.com/api/v3",
        api_key=api_key
    )
elif ak and sk:
    client = Ark(
        base_url="https://ark.cn-beijing.volces.com/api/v3",
        ak=ak,
        sk=sk
    )
else:
    raise ValueError("请设置 ARK_API_KEY 或 VOLC_ACCESS_KEY_ID / VOLC_SECRET_ACCESS_KEY")

# 你的 endpoint 名称
MODEL_NAME = "ep-20251117212117-9ztrl"

def test_ark():
    print("正在测试豆包 Ark API...")
    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": "测试一下你是否正常？"}],
        )
        txt = resp.choices[0].message.content
        print("✅ API 正常工作")
        print("返回内容：", txt)
    except Exception:
        print("❌ API 测试失败")
        traceback.print_exc()

test_ark()

# -----------------------------
# 2. 自定义任务（你在这里填）
# -----------------------------
chatbot_task = "帮我将用户输入翻译成专业学术英文"
prompt_for_task = (
    "You are an expert academic writing assistant. "
    "Your task is to rewrite the user's input into formal academic English. "
    "Do not add new ideas, only rewrite."
)


# -----------------------------
# 3. 重置会话
# -----------------------------
def reset() -> List:
    return []


# -----------------------------
# 4. 与豆包交互
# -----------------------------
def interact_customize(chatbot: List[Tuple[str, str]], prompt: str, user_input: str, temp=1.0):

    try:
        # 构建 messages
        messages = [{"role": "system", "content": prompt}]

        for a, b in chatbot:
            messages.append({"role": "user", "content": a})
            messages.append({"role": "assistant", "content": b})

        messages.append({"role": "user", "content": user_input})

        # 调用豆包
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=temp
        )

        # -------------------------
        # 兼容两类返回格式
        # -------------------------
        raw = resp.choices[0].message.content

        if isinstance(raw, str):
            answer = raw
        elif isinstance(raw, list):
            answer = "".join(c.get("text", "") for c in raw if isinstance(c, dict) and c.get("type") == "text")
        else:
            answer = str(raw)

        chatbot.append((user_input, answer))

    except Exception as e:
        chatbot.append((user_input, f"Error: {e}"))
        print("Error:", e)

    return chatbot


# -----------------------------
# 5. 导出结果
# -----------------------------
def export_customized(chatbot: List[Tuple[str, str]], description: str):
    target = {"chatbot": chatbot, "description": description}
    with open("part3.json", "w") as f:
        json.dump(target, f, ensure_ascii=False, indent=2)


# -----------------------------
# 6. Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("# Part3: Customized Task\nThe chatbot performs a specific task you designed.")

    chatbot = gr.Chatbot()

    desc_textbox = gr.Textbox(
        label="Description of the task",
        value=chatbot_task,
        interactive=False
    )

    prompt_textbox = gr.Textbox(
        label="Prompt",
        value=prompt_for_task,
        visible=False
    )

    input_textbox = gr.Textbox(label="Input")

    with gr.Column():
        gr.Markdown("### Temperature")
        temperature_slider = gr.Slider(0.0, 1.0, value=0.7, step=0.1)

    with gr.Row():
        send_button = gr.Button("Send")
        reset_button = gr.Button("Reset")

    with gr.Column():
        export_button = gr.Button("Export")

    send_button.click(
        interact_customize,
        inputs=[chatbot, prompt_textbox, input_textbox, temperature_slider],
        outputs=[chatbot]
    )

    reset_button.click(reset, outputs=[chatbot])

    export_button.click(export_customized, inputs=[chatbot, desc_textbox])

demo.launch(debug=True, share=True)
